# LDA

In [1]:
import pickle

with open('./datasets/small/HAS_e.pickle', 'rb') as f:
    HAS_e = pickle.load(f)

In [2]:
from time import time

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(embedding):
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=embedding['corpus'],
            id2word=embedding['id2word'],
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=embedding['T'], dictionary=embedding['id2word'], coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, embedding['id2word']))
        return (cs + ts) / 2

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()
    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [3]:
def save_models_lda(HAS_e):
    for v in HAS_e:
        model = lda(HAS_e[v])
        with open(f'./datasets/small/models_lda/{v}.pickle', 'wb') as f: # TODO: save using gensim save method
            pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)
        del model

In [4]:
save_models_lda(HAS_e)

[I 2023-08-17 10:07:01,774] A new study created in memory with name: no-name-f57b0248-a9d3-4f3d-b1bf-e32a619945ef
[I 2023-08-17 10:07:07,829] Trial 0 finished with value: 0.6600883153037636 and parameters: {'num_topics': 12, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'symmetric'}. Best is trial 0 with value: 0.6600883153037636.
[I 2023-08-17 10:07:14,335] Trial 1 finished with value: 0.6445837554562599 and parameters: {'num_topics': 23, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'symmetric'}. Best is trial 0 with value: 0.6600883153037636.


# BERTopic

In [ ]:
from time import time

from bertopic import BERTopic
from utils import get_topics_bertopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"
bertopic = BERTopic(embedding_model=embedding_model)

def save_models_bertopic(HAS_e):
    path_prefix = 'datasets/small/models_bertopic/'
    for v in HAS_e:
        t_start = time()
        model = bertopic.fit(HAS_e[v]['text'])
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        pd.DataFrame([{ 'time': t_end - t_start }]).to_csv(f'{path_prefix}{v}/time.csv', index=False)
        # print(get_topics_bertopic(model))
        del model

In [ ]:
save_models_bertopic(HAS_e)